In [22]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.chains import RetrievalQA

In [23]:
# 读取文本
from langchain.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("..\data\Virtual_characters.pdf")
PDF_data=loader.load()

In [24]:
# 切分文本
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=5)
all_splits = text_splitter.split_documents(PDF_data)

In [25]:

# 利用embedding模型建立向量数据库
persist_directory="db"
model_name = "all-MiniLM-L6-v2"
model_kwargs={"device":"cuda"}
embedding =HuggingFaceEmbeddings(model_name=model_name,model_kwargs=model_kwargs)
vectordb=Chroma.from_documents(documents=all_splits,embedding=embedding,persist_directory=persist_directory)

In [33]:
from transformers import AutoModelForCausalLM
model =AutoModelForCausalLM.from_pretrained('D:\Program Projects\Python Projects\DB-GPT\models\Qwen2-0.5B')
model=model.cuda()

loading configuration file D:\Program Projects\Python Projects\DB-GPT\models\Qwen2-0.5B\config.json
Model config Qwen2Config {
  "_name_or_path": "D:\\Program Projects\\Python Projects\\DB-GPT\\models\\Qwen2-0.5B",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151643,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 131072,
  "max_window_layers": 24,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.44.2",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

loading weights file D:\Program Projects\Python Projects\DB-GPT\models\Qwen2-0.5B\model.safetensors
Generate config GenerationC

In [36]:
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=model,
    chain_type = "stuff",
    retriever = retriever,
    verbose = True
)

ValidationError: 2 validation errors for LLMChain
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)